wr236596 statlist236596 fld236596a0
wr236839 statlist236839 fld236839a0

In [5]:
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import scipy.stats as st

pio.renderers.default = 'jupyterlab'

In [ ]:
DATA_PATH = os.path.join('data', 'meteo', 'wr236839')
DATA_METEOSTATIONS_PATH = os.path.join(DATA_PATH, 'statlist236839.txt')
DATA_FIELDS_PATH = os.path.join(DATA_PATH, 'fld236839a0.txt')
DATA_MAIN_PATH = os.path.join(DATA_PATH, 'wr236839.txt')

meteostations, headers = [], []

with open(DATA_METEOSTATIONS_PATH, encoding='WINDOWS-1251') as file:
    for line in file:
        idx, split_line = line.split(maxsplit=1)
        name, country = split_line.rsplit(maxsplit=1)
        meteostations.append((idx, name, country))

with open(DATA_FIELDS_PATH, encoding='WINDOWS-1251') as file:
    headers = [line.split(maxsplit=3)[3].rstrip() for line in file]

df_meteostations = pd.DataFrame(meteostations, columns=['Индекс ВМО', 'Название станции', 'Страна'])
df_main = pd.read_fwf(DATA_MAIN_PATH, names=headers)
df_main.rename(columns = {'Год': 'year', 'Месяц': 'month', 'День': 'day'}, inplace=True)
df_main.insert(1, 'Дата', pd.to_datetime(df_main[['year', 'month', 'day']], errors='coerce'))
df_main.drop(['year', 'month', 'day'], axis=1, inplace=True)
df_main.dropna(subset=['Дата'], inplace=True)
df_main.set_index('Дата')

In [ ]:
data = df_main.groupby(df_main['Дата'].dt.year)['Средняя температура воздуха'].mean()
data -= data.mean()
data = data.rolling(10, min_periods=10).mean().dropna()

idx, temperature = np.arange(0, data.size), data.array
res = st.linregress(idx, temperature)
line = res.slope * idx + res.intercept

layout = go.Layout(
    title='Скользящее среднее аномалии среднегодовой температуры воздуха (отклонение от средней за базовый период 2001-2022)',
    xaxis_title='Время, год',
    yaxis_title='Аномалия температуры, °C',
    xaxis_range=[2010, 2022],
    yaxis_range=[-0.4, 0.4]
)

fig = go.Figure(layout=layout)
fig.add_trace(go.Scatter(x=data.index, y=data.array, mode='markers', name='Аномалия среднегодовой температуры'))
fig.add_trace(go.Scatter(x=data.index, y=line, mode='lines', name='Аппроксимация'))
fig.show()
print(f'Рост температуры в среднем составляет {round(res.slope, 2)} °C за год.')

In [ ]:
data = df_main.groupby(df_main['Дата'].dt.year)['Количество осадков'].sum().rolling(10, min_periods=10).mean().dropna()

idx, precipitation = np.arange(0, data.size), data.array
res = st.linregress(idx, precipitation)
line = res.slope * idx + res.intercept

layout = go.Layout(
    title='Скользящее среднее по сумме количества осадков в каждом регионе за год',
    xaxis_title='Время, год',
    yaxis_title='Атмосферные осадки, мм',
    xaxis_range=[2010, 2022],
    yaxis_range=[155000, 175000]
)

fig = go.Figure(layout=layout)
fig.add_trace(go.Scatter(x=data.index, y=data.array, mode='markers', name='Количество осадков'))
fig.add_trace(go.Scatter(x=data.index, y=line, mode='lines', name='Аппроксимация'))
fig.show()
print(f'Падение объема осадков в среднем составляет {round(-res.slope, 2)} мм за год.')

In [8]:
DATA_PATH = os.path.join('data', 'meteo', 'fires')
DATA_FIELDS_PATH = os.path.join(DATA_PATH, 'codebook.csv')
DATA_MAIN_PATH = os.path.join(DATA_PATH, 'thermopoints.csv')

df_main = pd.read_csv(DATA_MAIN_PATH, sep=';', parse_dates=['dt'])
df_main['year'] = df_main['dt'].dt.year

In [ ]:
data = df_main

fig = px.scatter_mapbox(
    data,
    lat='lat',
    lon='lon',
    hover_name='type_name',
    hover_data=['dt', 'type_id'],
    color_discrete_sequence=['fuchsia'],
    animation_frame='year',
    zoom=3,
    height=600,
)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()
fig.show(renderer='browser')

In [ ]:
data = df_main.groupby(['year', 'type_name']).agg({'type_id': 'count'}).reset_index()

fig = px.bar(
    data,
    x='year',
    y='type_id',
    color='type_name',
    title='Количество пожаров на территории России за период 2012-2021',
    color_continuous_scale='Rainbow',
    labels={'year': 'Время, год',
            'type_id': 'Число пожаров',
            'type_name': 'Тип пожара'},
)
fig.show()